## All Imports

In [1]:
from func.helper import generate_embeddings,EmbeddingDataset,Generator,Discriminator
import numpy as np
import random as python_random
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as sklm
from sklearn.utils import shuffle
import numpy as np
import tensorflow as tf  # Import TensorFlow
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import torch.optim as optim
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from ast import literal_eval
import torch.nn.functional as F
import os 

/local/data0/software/miniconda3/envs/clam/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/local/data0/software/miniconda3/envs/clam/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/local/data0/software/miniconda3/envs/clam/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/local/data0/software/

## Model Configuration

In [2]:

# Check if CUDA is available and set PyTorch to use CUDA or CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset = EmbeddingDataset('balanced_df.csv.gz')


embedding_dim = 1376
race_dim = 2
hidden_dim = 512
batch_size = 512
lr = 0.00002
epochs = 300

data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
generator = Generator(embedding_dim, race_dim, hidden_dim).to(device)
discriminator = Discriminator(embedding_dim, race_dim, hidden_dim).to(device)

criterion = nn.BCELoss()

optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))


# Define the directory path
save_dir = 'savedweights'

# Create the directory if it does not exist
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

/local/data0/home/atauqeer/Negin/func/helper.py:366: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['race_index'] = filtered_data['race'].map(race_to_index)


## Training loop

In [ ]:
from torch.autograd import Variable

# Define the standard deviation for the noise
noise_std = 0.1

for epoch in range(epochs):
    for i, (real_embeddings, races) in enumerate(data_loader):
        if len(races) != batch_size:
            break
        real_embeddings = real_embeddings.to(device)
        races = F.one_hot(races, num_classes=2).to(device)
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # Train Discriminator
        optimizer_D.zero_grad()

        # Add noise to the real embeddings
        noisy_real_embeddings = real_embeddings + torch.randn_like(real_embeddings) * noise_std
        outputs = discriminator(noisy_real_embeddings, races)
        d_loss_real = criterion(outputs, real_labels)
        d_loss_real.backward()

        noise = torch.randn(batch_size, embedding_dim).to(device)
        fake_embeddings = generator(noise, races)

        # Add noise to the fake embeddings
        noisy_fake_embeddings = fake_embeddings + torch.randn_like(fake_embeddings) * noise_std
        outputs = discriminator(noisy_fake_embeddings.detach(), races)
        d_loss_fake = criterion(outputs, fake_labels)
        d_loss_fake.backward()

        optimizer_D.step()

        # Train Generator
        optimizer_G.zero_grad()

        outputs = discriminator(fake_embeddings, races)
        g_loss = criterion(outputs, real_labels)
        g_loss.backward()

        optimizer_G.step()

        if i % 100 == 0:
            print(f"Epoch [{epoch}/{epochs}] Batch {i}/{len(data_loader)} \
                  Loss D: {d_loss_real.item()+d_loss_fake.item()}, loss G: {g_loss.item()}")

    torch.save(generator.state_dict(), f'{save_dir}/generator_epoch_{epoch}.pt')
    torch.save(discriminator.state_dict(), f'{save_dir}/discriminator_epoch_{epoch}.pt')


## TEST Code : (generating 10 examples for race index 0)

In [ ]:
from func.helper import Generator, generate_embeddings
import torch
import torch.nn.functional as F

# Check if CUDA is available and set PyTorch to use CUDA or CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the dimensions and other parameters
embedding_dim = 1376
race_dim = 6
hidden_dim = 256
num_examples = 10
race_index = 0

# Initialize the generator model and move it to the appropriate device
generator = Generator(embedding_dim, race_dim, hidden_dim).to(device)
# Load the saved weights and move to the same device
generator.load_state_dict(torch.load('savedweights/generator_epoch_1.pt', map_location=device))
# Set the model to evaluation mode
generator.eval()

# Generate embeddings
generated_embeddings = generate_embeddings(generator, num_examples, race_index, embedding_dim)

print("Generated Embeddings:")
print(generated_embeddings)


Generated Embeddings:
tensor([[ 0.9673, -0.8214, -0.7557,  ..., -0.9629,  0.9245,  0.8999],
        [ 0.9771, -0.7713, -0.8164,  ..., -0.9757,  0.9681,  0.9172],
        [ 0.9357, -0.7177, -0.6751,  ..., -0.9326,  0.8680,  0.8434],
        ...,
        [ 0.9333, -0.6618, -0.7196,  ..., -0.9284,  0.9016,  0.8024],
        [ 0.8802, -0.6808, -0.5730,  ..., -0.9226,  0.7798,  0.8203],
        [ 0.8291, -0.5403, -0.5325,  ..., -0.8475,  0.8027,  0.6759]],
       device='cuda:0')


## Saving all examples in file

In [ ]:
import os
import torch
from func.helper import Generator, generate_embeddings

# Check if CUDA is available and set PyTorch to use CUDA or CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the dimensions and other parameters
embedding_dim = 1376
race_dim = 6
hidden_dim = 256
num_examples = 1000  # Increase the number of examples
race_index = 0

# Initialize the generator model and move it to the appropriate device
generator = Generator(embedding_dim, race_dim, hidden_dim).to(device)
# Load the saved weights and move to the same device
generator.load_state_dict(torch.load('/savedweights/generator_epoch_9.pt', map_location=device))
# Set the model to evaluation mode
generator.eval()

# Generate embeddings
generated_embeddings = []
batch_size = 100  # Adjust the batch size based on available memory
num_batches = num_examples // batch_size

with torch.no_grad():
    for _ in range(num_batches):
        embeddings_batch = generate_embeddings(generator, batch_size, race_index, embedding_dim)
        generated_embeddings.append(embeddings_batch)

# Concatenate batches into a single tensor
generated_embeddings = torch.cat(generated_embeddings, dim=0)

# Create directory if it doesn't exist
save_dir = "test_examples"
os.makedirs(save_dir, exist_ok=True)

# Save each embedding as a separate file
for i, embedding in enumerate(generated_embeddings):
    file_path = os.path.join(save_dir, f"embedding_{i}.pt")
    torch.save(embedding, file_path)

print("Embeddings saved successfully in the test_examples directory.")


# Making npz file for all the generated examples

import os
import numpy as np
import torch

# Directory containing embeddings
embeddings_dir = "test_examples"

# Initialize a list to store embeddings
embeddings = []

# Iterate over files in the directory
for filename in os.listdir(embeddings_dir):
    if filename.endswith(".pt"):
        # Load the embedding tensor
        file_path = os.path.join(embeddings_dir, filename)
        embedding = torch.load(file_path)
        embeddings.append(embedding.numpy())

# Stack embeddings into a single numpy array
combined_embeddings = np.stack(embeddings)

# Save combined embeddings to a NPZ file
np.savez("combined_embeddings.npz", embeddings=combined_embeddings)

print("Combined embeddings saved successfully as combined_embeddings.npz.")
